In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:02 of 0:12:02 remaining]
Chain 1:  10% [0:00:53 of 0:00:58 remaining]
Chain 1:  20% [0:00:54 of 0:01:08 remaining]
Chain 1:  30% [0:00:50 of 0:01:11 remaining]
Chain 1:  40% [0:00:43 of 0:01:11 remaining]
Chain 1:  50% [0:00:36 of 0:01:12 remaining]
Chain 1:  60% [0:00:30 of 0:01:15 remaining]
Chain 1:  70% [0:00:23 of 0:01:16 remaining]
Chain 1:  80% [0:00:15 of 0:01:16 remaining]
Chain 1:  90% [0:00:08 of 0:01:17 remaining]
Chain 1: 100% [0:00:00 of 0:01:17 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:49 of 0:00:54 remaining]
Chain 2:  20% [0:00:53 of 0:01:07 remaining]
Chain 2:  30% [0:00:50 of 0:01:12 remaining]
Chain 2:  40% [0:00:44 of 0:01:14 remaining]
Chain 2:  50% [0:00:37 of 0:01:15 remaining]
Chain 2:  60% [0:00:30 of 0:01:16 remaining]
Chain 2:  70% [0:00:23 of 0:01:16 remaining]
Chain 2:  80% [0:00:15 of 0:01:16 remaining]
Chain 2:  90% [0:00:08 of 0:01:16 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[26.2913 115.421 0.865979; 25.7002 115.992 0.853739; … ; 22.1981 111.206 0.952957; 23.9165 111.67 0.965405]

[22.473 112.121 0.948395; 30.124 112.121 0.948395; … ; 24.541 114.937 0.886813; 24.983 115.291 0.885434]

[23.1383 116.011 0.862418; 26.8815 116.167 0.858913; … ; 27.6613 112.489 0.933929; 24.5254 113.621 0.90432]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean          SD        Naive SE        MCSE        ESS   
     s2  26.17179629 1.983973898 0.01207408064 0.02028180127 9000.0000
beta[1] 113.88776279 1.902137790 0.01157604195 0.03473867134 2998.1789
beta[2]   0.90485865 0.041816994 0.00025449012 0.00075606107 3059.0873

Quantiles:
            2.5%        25.0%       50.0%      75.0%       97.5%  
     s2  22.5672435  24.80098522  26.062655  27.4303685  30.384891
beta[1] 110.0676231 112.61000199 113.894258 115.1957898 117.582864
beta[2]   0.8233703   0.87612724   0.904887   0.9328758   0.988582



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*